# yfinanceを使用した日本株式データ取得テストノートブック

このノートブックでは、`sumalize.py`で実際に使用している値の取得方法を解説しながらテストできます。

## 目的

- `sumalize.py`の`get_stock_data`関数で取得しているデータ項目を順を追って確認
- 各データ項目の取得方法
- テスト用のティッカーシンボルで実際のデータ取得

---

## テスト対象の設定


In [ ]:
import yfinance as yf
import pandas as pd
import requests
from datetime import timedelta

# ティッカーシンボルを指定（例: 7203.T = トヨタ自動車、7974.T = 任天堂、1828.T = サンヨーホームズ）
# sumalize.pyのformat_ticker関数と同じ形式: "コード.T"
ticker_symbol = "7974.T"  # 任天堂（データが豊富な銘柄）
ticker = yf.Ticker(ticker_symbol)

print(f"📊 テスト対象: {ticker_symbol}")
print("=" * 60)

---

## 1. 基本情報の取得

`ticker.info`から基本情報を取得します。

### 取得項目
- **会社名** (`longName`, `shortName`)
- **業種** (`industry`)
- **セクター** (`sector`)
- **郵便番号** (`zip`) - 後で都道府県に変換
- **現在の株価** (`currentPrice`)
- **時価総額** (`marketCap`)


In [ ]:
info = ticker.info

print("📋 【基本情報】")
print("-" * 60)
print(f"会社名（longName）: {info.get('longName', 'N/A')}")
print(f"会社名（shortName）: {info.get('shortName', 'N/A')}")
print(f"業種（industry）: {info.get('industry', 'N/A')}")
print(f"セクター（sector）: {info.get('sector', 'N/A')}")
print(f"郵便番号（zip）: {info.get('zip', 'N/A')}")

# 都道府県の取得（sumalize.pyのget_prefecture_from_zip関数と同じロジック）
zip_code = info.get('zip')
prefecture = None
if zip_code:
    try:
        clean_zip = str(zip_code).replace("-", "").replace("−", "").replace(" ", "").replace("　", "")
        if len(clean_zip) >= 7:
            url = f"https://digital-address.app/{clean_zip}"
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            if data.get("addresses") and len(data["addresses"]) > 0:
                prefecture = data["addresses"][0].get("pref_name")
    except Exception as e:
        print(f"  都道府県取得エラー: {e}")

print(f"都道府県: {prefecture if prefecture else 'N/A'}")
print(f"現在の株価（currentPrice）: {info.get('currentPrice', 'N/A')}")
if info.get('marketCap'):
    print(f"時価総額（marketCap）: {info.get('marketCap'):,.0f}")
else:
    print("時価総額: N/A")
print("=" * 60)

---

## 2. 市場データ（時価総額、PBR、PER）

市場で評価される指標を取得します。

### 取得項目
- **時価総額** (`marketCap`) - 株価 × 発行済み株式数
- **PBR** (`priceToBook`) - 株価純資産倍率
- **PER(会予)** (`forwardPE`) - 予想PER
- **PER直近** (`trailingPE`) - 過去12ヶ月の実績PER


In [ ]:
if 'info' not in locals():
    info = ticker.info

print("📈 【市場データ】")
print("-" * 60)
if info.get('marketCap'):
    print(f"時価総額（marketCap）: {info.get('marketCap'):,.0f}")
else:
    print("時価総額: N/A")
print(f"PBR（priceToBook）: {info.get('priceToBook', 'N/A')}")

# PER(会予) - 予想PER
forward_pe = info.get('forwardPE', None)
print(f"\nPER(会予)（forwardPE）: {forward_pe}")

# PER(過去12ヶ月) - 過去12ヶ月の実績PER
trailing_pe = info.get('trailingPE', None)
print(f"PER(過去12ヶ月)（trailingPE）: {trailing_pe}")

# sumalize.pyのcalculate_recent_pe関数と同じ計算方法
# PER = 現在の株価 / trailingEps（過去12ヶ月のEPS）
current_price = info.get('currentPrice') or info.get('regularMarketPrice')
trailing_eps = info.get('trailingEps')
if current_price and trailing_eps and trailing_eps != 0:
    calculated_pe = current_price / trailing_eps
    print(f"\n💡 計算したPER(過去12ヶ月)（currentPrice / trailingEps）: {calculated_pe:.2f}")
    if trailing_pe:
        diff = abs(calculated_pe - trailing_pe)
        print(f"   → trailingPEとの差: {diff:.4f}")

# PER(前年度)とEPS(前年度)は後で計算（財務諸表データが必要）
print("\nPER(前年度): 後で計算（セル10参照）")
print("=" * 60)

---

## 3. 配当関連データ

株主への配当に関する情報を取得します。

### 取得項目
- **配当方向性** (`payoutRatio`) - 配当性向（利益に対する配当の割合）
- **配当利回り** (`trailingAnnualDividendYield`) - 過去12ヶ月の配当利回り
- **配当金履歴** (`ticker.dividends`) - 過去の配当額の推移



In [ ]:
if 'info' not in locals():
    info = ticker.info

print("💰 【配当関連データ】")
print("-" * 60)

# 配当方向性（配当性向）- sumalize.pyでは「配当方向性」として保存
payout_ratio = info.get('payoutRatio', None)
print(f"配当方向性（payoutRatio）: {payout_ratio}")
if payout_ratio:
    print(f"  → 配当性向: {payout_ratio * 100:.2f}%")

# 配当利回り（過去12ヶ月）
dividend_yield = info.get('trailingAnnualDividendYield', None)
print(f"\n配当利回り（trailingAnnualDividendYield）: {dividend_yield}")
if dividend_yield:
    print(f"  → 配当利回り: {dividend_yield * 100:.2f}%")

# 配当金履歴（実際の配当額の推移）
dividends = ticker.dividends
if not dividends.empty:
    print("\n📅 直近の配当金履歴（最新5件）:")
    print(dividends.tail(5))
    latest_dividend = dividends.iloc[-1]
    print(f"\n最新配当額: {latest_dividend}円")
else:
    print("\n配当金履歴: データなし")
print("=" * 60)

---

## 4. EPS関連データ

1株あたりの利益（Earnings Per Share）を取得します。

### 取得項目
- **EPS(過去12ヶ月)** (`trailingEps`) - 過去12ヶ月の実績EPS
- **EPS(予想)** (`forwardEps`) - 予想EPS



In [ ]:
if 'info' not in locals():
    info = ticker.info

print("📊 【EPS関連データ】")
print("-" * 60)

# EPS(過去12ヶ月) - 実績EPS
trailing_eps = info.get('trailingEps', None)
print(f"EPS(過去12ヶ月)（trailingEps）: {trailing_eps}")

# EPS(予想) - 予想EPS
forward_eps = info.get('forwardEps', None)
print(f"EPS(予想)（forwardEps）: {forward_eps}")

# EPS(前年度)は後で計算（財務諸表データが必要）
print("\nEPS(前年度): 後で計算（セル10参照）")
print("=" * 60)

---

## 5. 収益性指標

企業の収益性を評価する指標を取得します。

### 取得項目
- **ROE** (`returnOnEquity`) - 自己資本利益率
- **営業利益率** (`operatingMargins`) - 売上に対する営業利益の割合
- **純利益率** (`profitMargins`) - 売上に対する純利益の割合


In [ ]:
if 'info' not in locals():
    info = ticker.info

print("📈 【収益性指標】")
print("-" * 60)

# ROE（自己資本利益率）
roe = info.get('returnOnEquity', None)
print(f"ROE（returnOnEquity）: {roe}")
if roe:
    print(f"  → ROE: {roe * 100:.2f}%")

# 営業利益率
operating_margins = info.get('operatingMargins', None)
print(f"\n営業利益率（operatingMargins）: {operating_margins}")
if operating_margins:
    print(f"  → 営業利益率: {operating_margins * 100:.2f}%")

# 純利益率
profit_margins = info.get('profitMargins', None)
print(f"\n純利益率（profitMargins）: {profit_margins}")
if profit_margins:
    print(f"  → 純利益率: {profit_margins * 100:.2f}%")
print("=" * 60)


---

## 6. 財務諸表データ（損益計算書）

損益計算書（Income Statement）から主要な財務データを取得します。

### 取得項目
- **売上高** (`Total Revenue`)
- **営業利益** (`Operating Income`)
- **当期純利益** (`Net Income`)


In [ ]:
print("📑 【財務諸表データ（損益計算書）】")
print("-" * 60)

try:
    financials = ticker.financials
    
    if not financials.empty:
        print("📊 財務諸表の構造:")
        print(f"  決算期数: {len(financials.columns)}")
        print(f"  項目数: {len(financials.index)}")
        print("\n📅 決算期一覧（最新から）:")
        for i, col in enumerate(financials.columns[:5]):  # 最新5期のみ表示
            print(f"  {i+1}. {col}")
        
        # 最新決算期（最初の列）を取得
        latest_period = financials.columns[0]
        print(f"\n最新決算期: {latest_period}")
        
        # sumalize.pyと同じ方法でデータを取得
        def safe_get_financial_item(df, item_name, fallback_items=None):
            """sumalize.pyのsafe_get_financial_data関数と同じロジック"""
            if item_name in df.index:
                value = df.loc[item_name, latest_period]
                return value if pd.notna(value) else None
            
            if fallback_items:
                for fallback in fallback_items:
                    if fallback in df.index:
                        value = df.loc[fallback, latest_period]
                        return value if pd.notna(value) else None
            return None
        
        # 売上高
        total_revenue = safe_get_financial_item(financials, "Total Revenue")
        if total_revenue:
            print(f"\n💰 売上高（Total Revenue）: {total_revenue:,.0f}")
        else:
            print("\n売上高: N/A")
        
        # 営業利益
        operating_income = safe_get_financial_item(financials, "Operating Income")
        if operating_income:
            print(f"💵 営業利益（Operating Income）: {operating_income:,.0f}")
        else:
            print("営業利益: N/A")
        
        # 当期純利益
        net_income = safe_get_financial_item(financials, "Net Income")
        if net_income:
            print(f"💴 当期純利益（Net Income）: {net_income:,.0f}")
        else:
            print("当期純利益: N/A")
        
        # 営業利益率の計算（売上高に対する営業利益の割合）
        if total_revenue and operating_income and total_revenue != 0:
            operating_margin = operating_income / total_revenue
            print(f"\n📊 営業利益率（計算値）: {operating_margin:.4f} ({operating_margin * 100:.2f}%)")
            print(f"   = 営業利益 / 売上高 = {operating_income:,.0f} / {total_revenue:,.0f}")
        
        # 純利益率の計算（売上高に対する当期純利益の割合）
        if total_revenue and net_income and total_revenue != 0:
            net_margin = net_income / total_revenue
            print(f"\n📊 純利益率（計算値）: {net_margin:.4f} ({net_margin * 100:.2f}%)")
            print(f"   = 当期純利益 / 売上高 = {net_income:,.0f} / {total_revenue:,.0f}")
        
        # 財務諸表の一部を表示
        print("\n📋 財務諸表データ（主要項目、最新決算期）:")
        key_items = ["Total Revenue", "Operating Income", "Net Income", "Total Operating Expenses", "Diluted Average Shares"]
        for item in key_items:
            if item in financials.index:
                value = financials.loc[item, latest_period]
                if pd.notna(value):
                    print(f"  • {item}: {value:,.0f}")
    else:
        print("❌ 財務諸表データが取得できませんでした")
        
except Exception as e:
    print(f"❌ 財務諸表取得エラー: {e}")

print("=" * 60)


---

## 7. バランスシートデータ

貸借対照表（Balance Sheet）から財務健全性に関するデータを取得します。

### 取得項目
- **負債** (`Total Liabilities Net Minority Interest` / `Total Liab`)
- **流動負債** (`Current Liabilities` / `Total Current Liabilities`)
- **流動資産** (`Current Assets` / `Total Current Assets`)
- **総負債** (`Total Debt`)
- **現金及び現金同等物** (`Cash And Cash Equivalents`)
- **投資有価証券** (`Available For Sale Securities` / `Short Term Investments`)
- **自己資本** (`Stockholders Equity` / `Total Stockholder Equity`)
- **総資産** (`Total Assets`)

In [ ]:
print("📊 【バランスシートデータ】")
print("-" * 60)

try:
    balance_sheet = ticker.balance_sheet
    
    if not balance_sheet.empty:
        print("📋 バランスシートの構造:")
        print(f"  決算期数: {len(balance_sheet.columns)}")
        print(f"  項目数: {len(balance_sheet.index)}")
        
        # 最新決算期（最初の列）を取得
        latest_period = balance_sheet.columns[0]
        print(f"\n最新決算期: {latest_period}")
        
        # sumalize.pyと同じ方法でデータを取得（フォールバック付き）
        def safe_get_balance_item(df, item_name, fallback_items=None):
            """sumalize.pyのsafe_get_financial_data関数と同じロジック"""
            if item_name in df.index:
                value = df.loc[item_name, latest_period]
                return value if pd.notna(value) else None
            
            if fallback_items:
                for fallback in fallback_items:
                    if fallback in df.index:
                        value = df.loc[fallback, latest_period]
                        return value if pd.notna(value) else None
            return None
        
        # 負債（フォールバック付き）
        total_liabilities = safe_get_balance_item(
            balance_sheet, 
            "Total Liabilities Net Minority Interest",
            fallback_items=["Total Liab"]
        )
        if total_liabilities:
            print(f"\n💳 負債（Total Liabilities）: {total_liabilities:,.0f}")
        else:
            print("\n負債: N/A")
        
        # 流動負債（フォールバック付き）
        current_liabilities = safe_get_balance_item(
            balance_sheet,
            "Current Liabilities",
            fallback_items=["Total Current Liabilities"]
        )
        if current_liabilities:
            print(f"💳 流動負債（Current Liabilities）: {current_liabilities:,.0f}")
        else:
            print("流動負債: N/A")
        
        # 流動資産（フォールバック付き）
        current_assets = safe_get_balance_item(
            balance_sheet,
            "Current Assets",
            fallback_items=["Total Current Assets"]
        )
        if current_assets:
            print(f"💵 流動資産（Current Assets）: {current_assets:,.0f}")
        else:
            print("流動資産: N/A")
        
        # 総負債
        total_debt = safe_get_balance_item(balance_sheet, "Total Debt")
        if total_debt:
            print(f"💳 総負債（Total Debt）: {total_debt:,.0f}")
        else:
            print("総負債: N/A")
        
        # 現金及び現金同等物（フォールバック付き）
        cash_and_equivalents = safe_get_balance_item(
            balance_sheet,
            "Cash And Cash Equivalents",
            fallback_items=["Cash Cash Equivalents And Short Term Investments"]
        )
        if cash_and_equivalents:
            print(f"💰 現金及び現金同等物: {cash_and_equivalents:,.0f}")
        else:
            print("現金及び現金同等物: N/A")
        
        # 投資有価証券（フォールバック付き）
        investments = safe_get_balance_item(
            balance_sheet,
            "Available For Sale Securities",
            fallback_items=["Short Term Investments", "Investmentin Financial Assets"]
        )
        if investments:
            print(f"📈 投資有価証券: {investments:,.0f}")
        else:
            print("投資有価証券: N/A")
        
        # 自己資本（フォールバック付き）
        total_equity = safe_get_balance_item(
            balance_sheet,
            "Stockholders Equity",
            fallback_items=["Total Stockholder Equity"]
        )
        if total_equity:
            print(f"💼 自己資本（Stockholders Equity）: {total_equity:,.0f}")
        else:
            print("自己資本: N/A")
        
        # 総資産
        total_assets = safe_get_balance_item(balance_sheet, "Total Assets")
        if total_assets:
            print(f"🏢 総資産（Total Assets）: {total_assets:,.0f}")
        else:
            print("総資産: N/A")
        
    else:
        print("❌ バランスシートデータが取得できませんでした")
        
except Exception as e:
    print(f"❌ バランスシート取得エラー: {e}")

print("=" * 60)


---

## 8. 計算項目

取得したデータから、追加の指標を計算します。

### 計算項目
1. **自己資本比率** = 自己資本 / 総資産
2. **ネットキャッシュ** = 流動資産 + (投資有価証券 × 0.7) - 負債
3. **ネットキャッシュ比率** = ネットキャッシュ / 時価総額

In [ ]:
print("🧮 【計算項目】")
print("-" * 60)

# 前のセルで取得したデータを使用（変数が定義されている場合）
try:
    # 自己資本比率の計算
    if 'total_equity' in locals() and 'total_assets' in locals():
        if total_equity and total_assets:
            equity_ratio = total_equity / total_assets
            print("📊 自己資本比率 = 自己資本 / 総資産")
            print(f"  = {total_equity:,.0f} / {total_assets:,.0f}")
            print(f"  = {equity_ratio:.4f} ({equity_ratio * 100:.2f}%)")
        else:
            print("❌ 自己資本比率: 計算不可（データ不足）")
    else:
        print("⚠️  自己資本比率: 前のセル（セル7）を実行してください")
    
    # ネットキャッシュの計算（sumalize.pyのcalculate_net_cash関数と同じロジック）
    if 'current_assets' in locals() and 'total_liabilities' in locals():
        if current_assets is not None and total_liabilities is not None:
            # 投資有価証券は70%で評価（保守的な見積もり）
            inv_value = (investments * 0.7) if 'investments' in locals() and investments is not None else 0
            net_cash = current_assets + inv_value - total_liabilities
            
            print("\n💰 ネットキャッシュ = 流動資産 + (投資有価証券 × 0.7) - 負債")
            if 'investments' in locals() and investments:
                print(f"  = {current_assets:,.0f} + ({investments:,.0f} × 0.7) - {total_liabilities:,.0f}")
                print(f"  = {current_assets:,.0f} + {inv_value:,.0f} - {total_liabilities:,.0f}")
            else:
                print(f"  = {current_assets:,.0f} + 0 - {total_liabilities:,.0f}")
            print(f"  = {net_cash:,.0f}")
            
            # ネットキャッシュ比率の計算
            if 'info' not in locals():
                info = ticker.info
            market_cap = info.get('marketCap')
            if net_cash and market_cap:
                net_cash_ratio = net_cash / market_cap
                print("\n📈 ネットキャッシュ比率 = ネットキャッシュ / 時価総額")
                print(f"  = {net_cash:,.0f} / {market_cap:,.0f}")
                print(f"  = {net_cash_ratio:.4f} ({net_cash_ratio * 100:.2f}%)")
            else:
                print("\n❌ ネットキャッシュ比率: 計算不可（時価総額データ不足）")
        else:
            print("\n❌ ネットキャッシュ: 計算不可（データ不足）")
    else:
        print("\n⚠️  ネットキャッシュ: 前のセル（セル7）を実行してください")
    
except NameError as e:
    print("❌ 計算エラー: 必要なデータが取得できていません。前のセルを実行してください。")
    print(f"  エラー詳細: {e}")

print("=" * 60)


---

## 9. 決算月の取得

バランスシートの最新決算期から決算月を取得します。

### 取得方法
- バランスシートの最新決算期（最初の列）から日付を抽出
- datetimeオブジェクトの場合は`strftime`でフォーマット
- 文字列の場合は時間部分を削除


In [ ]:
print("📅 【決算月の取得】")
print("-" * 60)

try:
    balance_sheet = ticker.balance_sheet
    
    if not balance_sheet.empty:
        cols = balance_sheet.columns.tolist()
        if cols:
            # 最新決算期から日付部分のみ抽出（sumalize.pyと同じロジック）
            latest_period = cols[0]
            
            if hasattr(latest_period, "strftime"):
                # datetimeオブジェクトの場合、日付部分のみ取得
                settlement_period = latest_period.strftime("%Y-%m-%d")
            else:
                # 文字列の場合、時間部分を削除
                settlement_period = str(latest_period).split(" ")[0]
            
            print(f"最新決算期（生データ）: {latest_period}")
            print(f"決算月（フォーマット後）: {settlement_period}")
        else:
            print("❌ 決算期データがありません")
    else:
        print("❌ 決算月: データ取得不可")
        
except Exception as e:
    print(f"❌ 決算月取得エラー: {e}")

print("=" * 60)


---

## 10. 前年度のPERとEPSの計算

前年度のPERとEPSを計算します（sumalize.pyの`calculate_previous_year_per`関数と同じロジック）。

### 計算方法
1. 財務諸表の2番目の列（前年度）からNet IncomeとDiluted Average Sharesを取得
2. 前年度EPS = 前年度Net Income / 前年度Diluted Average Shares
3. 前年度末の株価を取得（決算期の前後数日で取得）
4. 前年度PER = 前年度末株価 / 前年度EPS

### 取得項目
- **PER(前年度)** - 前年度末の株価と前年度のEPSから計算
- **EPS(前年度)** - 前年度のNet IncomeとDiluted Average Sharesから計算


In [ ]:
print("📊 【前年度のPERとEPSの計算】")
print("-" * 60)

try:
    if 'financials' not in locals():
        financials = ticker.financials
    
    if financials.empty:
        print("❌ 財務諸表データが取得できませんでした")
    else:
        cols = financials.columns.tolist()
        if len(cols) < 2:
            print("❌ 前年度のデータが存在しません（決算期が1期のみ）")
        else:
            # 前年度の決算期を取得（2番目の列）
            previous_year_period = cols[1]
            print(f"前年度決算期: {previous_year_period}")
            
            # 前年度のNet Incomeを取得
            net_income_key = "Net Income"
            if net_income_key not in financials.index:
                print("❌ Net Incomeが見つかりません")
            else:
                net_income_last_year = financials.loc[net_income_key, previous_year_period]
                if pd.isna(net_income_last_year) or net_income_last_year == 0:
                    print("❌ 前年度のNet Incomeが取得できませんでした")
                else:
                    print(f"\n💰 前年度Net Income: {net_income_last_year:,.0f}")
                    
                    # 前年度のDiluted Average Sharesを取得
                    shares_key = "Diluted Average Shares"
                    if shares_key not in financials.index:
                        print("❌ Diluted Average Sharesが見つかりません")
                    else:
                        shares_last_year = financials.loc[shares_key, previous_year_period]
                        if pd.isna(shares_last_year) or shares_last_year == 0:
                            print("❌ 前年度のDiluted Average Sharesが取得できませんでした")
                        else:
                            print(f"📊 前年度Diluted Average Shares: {shares_last_year:,.0f}")
                            
                            # 前年度EPSを計算
                            eps_last_year = net_income_last_year / shares_last_year
                            print(f"\n💡 前年度EPS = Net Income / Diluted Average Shares")
                            print(f"   = {net_income_last_year:,.0f} / {shares_last_year:,.0f}")
                            print(f"   = {eps_last_year:.2f}")
                            
                            # 前年度末の日付を取得
                            if hasattr(previous_year_period, "to_pydatetime"):
                                previous_year_date = previous_year_period.to_pydatetime()
                            elif hasattr(previous_year_period, "strftime"):
                                previous_year_date = previous_year_period
                            else:
                                try:
                                    previous_year_date = pd.to_datetime(previous_year_period).to_pydatetime()
                                except:
                                    previous_year_date = None
                            
                            if previous_year_date:
                                # 前年度末の株価を取得（決算期の前後数日で取得）
                                start_date = previous_year_date - timedelta(days=3)
                                end_date = previous_year_date + timedelta(days=3)
                                
                                try:
                                    price_history = ticker.history(
                                        start=start_date.strftime("%Y-%m-%d"), 
                                        end=end_date.strftime("%Y-%m-%d")
                                    )
                                    if price_history.empty:
                                        print("\n❌ 前年度末の株価が取得できませんでした")
                                    else:
                                        price_last_year = price_history["Close"].iloc[0]
                                        if pd.isna(price_last_year) or price_last_year == 0:
                                            print("\n❌ 前年度末の株価が無効です")
                                        else:
                                            print(f"\n💹 前年度末株価: {price_last_year:,.0f}円")
                                            
                                            # 前年度PERを計算
                                            per_last_year = price_last_year / eps_last_year
                                            print(f"\n📈 前年度PER = 前年度末株価 / 前年度EPS")
                                            print(f"   = {price_last_year:,.0f} / {eps_last_year:.2f}")
                                            print(f"   = {per_last_year:.2f}")
                                            
                                            # 変数に保存（後で使用）
                                            previous_year_pe = per_last_year
                                            previous_year_eps = eps_last_year
                                except Exception as e:
                                    print(f"\n❌ 前年度株価取得エラー: {e}")
                            else:
                                print("\n❌ 前年度の日付が取得できませんでした")
except Exception as e:
    print(f"❌ 前年度PER/EPS計算エラー: {e}")

print("=" * 60)


---

## 11. 全データ項目のサマリー

これまでに取得したすべてのデータ項目を一覧表示します。

### 取得データ項目一覧

**基本情報:**
- 会社名、銘柄コード、業種、優先市場、決算月、都道府県

**市場データ:**
- 時価総額、PBR、PER(会予)、PER(過去12ヶ月)、PER(前年度)

**配当関連:**
- 配当方向性、配当利回り

**EPS関連:**
- EPS(過去12ヶ月)、EPS(予想)、EPS(前年度)

**収益性指標:**
- ROE、営業利益率、純利益率

**財務諸表（損益計算書）:**
- 売上高、営業利益、当期純利益

**バランスシート:**
- 負債、流動負債、流動資産、総負債、現金及び現金同等物、投資有価証券、自己資本、総資産

**計算項目:**
- 自己資本比率、ネットキャッシュ、ネットキャッシュ比率


In [ ]:
print("📋 【全データ項目のサマリー】")
print("=" * 60)

# すべてのデータを辞書形式でまとめる
summary_data = {}

# 基本情報
if 'info' not in locals():
    info = ticker.info

summary_data["基本情報"] = {
    "会社名": info.get('longName') or info.get('shortName'),
    "銘柄コード": ticker_symbol.replace('.T', ''),
    "業種": info.get('industry') or info.get('sector'),
    "都道府県": prefecture if 'prefecture' in locals() else None,
}

# 決算月
try:
    if 'balance_sheet' not in locals():
        balance_sheet = ticker.balance_sheet
    if not balance_sheet.empty:
        latest_period = balance_sheet.columns[0]
        if hasattr(latest_period, "strftime"):
            settlement_period = latest_period.strftime("%Y-%m-%d")
        else:
            settlement_period = str(latest_period).split(" ")[0]
        summary_data["基本情報"]["決算月"] = settlement_period
except:
    summary_data["基本情報"]["決算月"] = None

# 市場データ
summary_data["市場データ"] = {
    "時価総額": info.get('marketCap'),
    "PBR": info.get('priceToBook'),
    "PER(会予)": info.get('forwardPE'),
    "PER(過去12ヶ月)": info.get('trailingPE'),
    "PER(前年度)": previous_year_pe if 'previous_year_pe' in locals() else None,
}

# 配当関連
summary_data["配当関連"] = {
    "配当方向性": info.get('payoutRatio'),
    "配当利回り": info.get('trailingAnnualDividendYield'),
}

# EPS関連
summary_data["EPS関連"] = {
    "EPS(過去12ヶ月)": info.get('trailingEps'),
    "EPS(予想)": info.get('forwardEps'),
    "EPS(前年度)": previous_year_eps if 'previous_year_eps' in locals() else None,
}

# 収益性指標
summary_data["収益性指標"] = {
    "ROE": info.get('returnOnEquity'),
    "営業利益率": info.get('operatingMargins'),
    "純利益率": info.get('profitMargins'),
}

# 財務諸表
try:
    if 'financials' not in locals():
        financials = ticker.financials
    if not financials.empty:
        latest_period = financials.columns[0]
        def safe_get(item):
            if item in financials.index:
                value = financials.loc[item, latest_period]
                return value if pd.notna(value) else None
            return None
        
        summary_data["財務諸表"] = {
            "売上高": safe_get("Total Revenue"),
            "営業利益": safe_get("Operating Income"),
            "当期純利益": safe_get("Net Income"),
        }
    else:
        summary_data["財務諸表"] = {"売上高": None, "営業利益": None, "当期純利益": None}
except:
    summary_data["財務諸表"] = {"売上高": None, "営業利益": None, "当期純利益": None}

# バランスシート
try:
    if 'balance_sheet' not in locals():
        balance_sheet = ticker.balance_sheet
    if not balance_sheet.empty:
        latest_period = balance_sheet.columns[0]
        def safe_get_bs(item, fallback=None):
            if item in balance_sheet.index:
                value = balance_sheet.loc[item, latest_period]
                if pd.notna(value):
                    return value
            if fallback and fallback in balance_sheet.index:
                value = balance_sheet.loc[fallback, latest_period]
                return value if pd.notna(value) else None
            return None
        
        total_liabilities = safe_get_bs("Total Liabilities Net Minority Interest", "Total Liab")
        current_liabilities = safe_get_bs("Current Liabilities", "Total Current Liabilities")
        current_assets = safe_get_bs("Current Assets", "Total Current Assets")
        total_equity = safe_get_bs("Stockholders Equity", "Total Stockholder Equity")
        total_assets = safe_get_bs("Total Assets")
        total_debt = safe_get_bs("Total Debt")
        cash_and_equivalents = safe_get_bs("Cash And Cash Equivalents", "Cash Cash Equivalents And Short Term Investments")
        investments = safe_get_bs("Available For Sale Securities", "Short Term Investments")
        
        summary_data["バランスシート"] = {
            "負債": total_liabilities,
            "流動負債": current_liabilities,
            "流動資産": current_assets,
            "総負債": total_debt,
            "現金及び現金同等物": cash_and_equivalents,
            "投資有価証券": investments,
            "自己資本": total_equity,
            "総資産": total_assets,
        }
        
        # 計算項目
        equity_ratio = (total_equity / total_assets) if (total_equity and total_assets) else None
        inv_value = (investments * 0.7) if investments else 0
        net_cash = (current_assets + inv_value - total_liabilities) if (current_assets is not None and total_liabilities is not None) else None
        market_cap = info.get('marketCap')
        net_cash_ratio = (net_cash / market_cap) if (net_cash and market_cap) else None
        
        summary_data["計算項目"] = {
            "自己資本比率": equity_ratio,
            "ネットキャッシュ": net_cash,
            "ネットキャッシュ比率": net_cash_ratio,
        }
    else:
        summary_data["バランスシート"] = {}
        summary_data["計算項目"] = {}
except:
    summary_data["バランスシート"] = {}
    summary_data["計算項目"] = {}

# サマリーを表示
for category, data in summary_data.items():
    print(f"\n📊 【{category}】")
    print("-" * 60)
    for key, value in data.items():
        if value is None:
            print(f"{key}: N/A")
        elif isinstance(value, (int, float)):
            if abs(value) >= 1e9:
                print(f"{key}: {value:,.0f}")
            elif abs(value) >= 1e6:
                print(f"{key}: {value:,.0f}")
            elif abs(value) >= 1e3:
                print(f"{key}: {value:,.2f}")
            else:
                print(f"{key}: {value:.4f}")
        else:
            print(f"{key}: {value}")

print("\n" + "=" * 60)
print("✅ 全データ項目の取得が完了しました")
print("=" * 60)
